## The OpenAI API

In [8]:
# https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety
# https://platform.openai.com/docs/api-reference/authentication
import os
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
openai.organization = "org-mGi0K4AmKu41HxKowsVRjoNh"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

In [5]:
# Let's test our connection to openAI
llm.invoke("What's the world's funniest joke? Has there ever been any scientific analysis?")

'\n\nThe world’s funniest joke is a subjective matter and there has not been any scientific analysis conducted on the topic. What one person may find funny, another may find dull or even offensive.'

## LangChain

In [6]:
llm.invoke("What are the LangChain Conversational agents?")

'\n\nLangChain Conversational agents are artificial intelligence-based conversational agents that are designed to simulate human conversation. They are powered by natural language processing and machine learning technologies, enabling them to understand and respond to user input. They can be used for a range of applications, from customer service and support to providing personalized recommendations.'

In [9]:
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))
chat_model = ChatOpenAI()

In [191]:
text = "What would be a good company name for a company that makes pacemakers?"
llm.predict(text)

'\n\nPaceLife Technologies'

In [192]:
chat_model.predict(text)

'HeartSync Technologies'

### PromptTemplate + LLM

In [199]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template("List all ICD10 code for {foo}.")
chain = prompt | llm | StrOutputParser()
chain.invoke({"foo": "septic shock"})

'\n\nA41.0 - Septic shock with acute organ dysfunction\nA41.1 - Septic shock with acute renal failure\nA41.2 - Septic shock with acute respiratory failure\nA41.3 - Septic shock with acute cardiorespiratory failure\nA41.4 - Septic shock with multiple organ dysfunction syndrome\nA41.9 - Septic shock, unspecified'

### Retrieval augmented generation (RAG) with Longchain

In [27]:
from operator import itemgetter
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

In [201]:
llm.invoke("What is the FAISS library?")

'\n\nFAISS (Facebook Artificial Intelligence Similarity Search) is an open source library for efficient similarity search and clustering of high-dimensional data. It was developed by Facebook AI Research and is written in C++ with complete wrappers for Python. FAISS allows for the efficient storage and retrieval of vectors in large datasets, making it useful for tasks such as large-scale information retrieval, recommendation systems, document search, and clustering.'

In [202]:
clinical_note = "A 28-year-old previously healthy adult patient presented with tachycardia, fever, and mental confusion. The symptoms started after a cut to his leg while gardening."
vectorstore = FAISS.from_texts([clinical_note], 
                               embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever() # this implements a RAG

template =  """Answer the question based only on the following context:
            {context}
            Question: {question}
            """
prompt = ChatPromptTemplate.from_template(template)

In [203]:
# This code converts the question and context into a prompt, passes it to the llm, and parses the llm output 
chain = ( {"context": retriever, "question": RunnablePassthrough()} | prompt  | llm | StrOutputParser() )

In [204]:
chain.invoke("When did this patient get his cut?")

'\nAnswer: The patient got his cut while gardening.'

In [205]:
chain.invoke("What is the location of the cut?")

"\nAnswer: The cut was to the patient's leg."

In [211]:
# Now let's try to see if we can get the answers in another language!

template =  """Answer the question based only on the following context:
            {context}
            Question: {question}
            Answer in the following language: {language}
            """
prompt = ChatPromptTemplate.from_template(template)

chain = { "context": itemgetter("question") | retriever, "question": itemgetter("question"), "language": itemgetter("language")} | prompt | llm | StrOutputParser()

In [212]:
chain.invoke({"question": "When did this patient get his cut?", "language": "spanish"})

'\nEl paciente recibió su corte después de jardinería.'

In [213]:
chain.invoke({"question": "What is the location of the cut?", "language": "spanish"})

'\nEl corte está en la pierna.'

### RAG with API calls: A Wikipedia Example

In [214]:
from wikipediaapi import Wikipedia
wiki = Wikipedia('ShamimBot/0.0', 'en')
jh_page = wiki.page('Social_determinants_of_health').text
jh_page = jh_page.split('\nReferences\n')[0]

In [215]:
print(jh_page[:500])

The  social determinants of health (SDOH) are the economic and social conditions that influence individual and group differences in health status. They are the health promoting factors found in one's living and working conditions (such as the distribution of income, wealth, influence, and power), rather than individual risk factors (such as behavioral risk factors or genetics) that influence the risk for a disease, or vulnerability to disease or injury. The distributions of social determinants a


In [216]:
# we are now going to store the wikipedia page in our vector store and use it as needed to answer questions
vectorstore = FAISS.from_texts([jh_page[:500]],  embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
              {context}
              Question: {question}
            """
prompt = ChatPromptTemplate.from_template(template)

In [217]:
chain = ( {"context": retriever, "question": RunnablePassthrough()} | prompt  | llm | StrOutputParser() )

In [218]:
chain.invoke("What are some health promoting factors?")

'\nAnswer: Health promoting factors can include the distribution of income, wealth, influence, and power.'

### Running Arbitrary Functions

In [222]:
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
chain = {"a": itemgetter("arg1") | RunnableLambda(length_function), "b": itemgetter("arg2") | RunnableLambda(length_function) } | prompt | llm | StrOutputParser()

In [223]:
chain.invoke({"arg1": "patient","arg2":"abx"})

'\n\nRobot: 10'

### Agents and Tools!!!

In [225]:
from langchain.agents import load_tools

tools = load_tools( ["llm-math"], llm=model )

template =  """You're an ED doctor. Your patient is 85kg.
              Question: {question}
            """
prompt = ChatPromptTemplate.from_template(template)
chain = ( {"question": RunnablePassthrough()}  | prompt | llm  | StrOutputParser())
chain.invoke("How many liters of fluids my patient will receive if administered 30 mL/kg/hour IV infusion over 3 hours?")

"To calculate the total amount of fluids your patient will receive, you need to multiply the infusion rate (30 mL/kg/hour) by the patient's weight (85 kg) and the duration of the infusion (3 hours).\n\nTotal amount of fluids = Infusion rate × Weight × Duration\n\nTotal amount of fluids = 30 mL/kg/hour × 85 kg × 3 hours\n\nTotal amount of fluids = 7650 mL\n\nTherefore, your patient will receive 7650 mL (or 7.65 liters) of fluids during the 3-hour IV infusion."

In [ ]:
# What happens if patient weight is unknown?
model.invoke("How many liters of fluids a patient will receive if administered 30 mL/kg/hour IV infusion over 3 hours?")

In [226]:
template = """You're an ED doctor.
              {context}
              Question: {question}
            """
prompt = ChatPromptTemplate.from_template(template)
chain = ( {"context": retriever, "question": RunnablePassthrough()}  | prompt | llm  | StrOutputParser())
chain.invoke("How many liters of fluids my patient will receive if administered 30 mL/kg/hour IV infusion over 3 hours?") # What is the patient weight? RAG to come to rescue

"To calculate the total amount of fluid your patient will receive, we need to know their weight. Please provide the patient's weight in kilograms."

#### Let's now build an agent that can use tools!

In [229]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

In [251]:
vectorstore = FAISS.from_texts([clinical_note,"Temperature=99, Pulse=110, BP=100/60, Respiration=24, SpO2=92%", 'body weight = 85 kg', "redness, warmth, tenderness, and swelling of the skin"], 
                               embedding=OpenAIEmbeddings())
llm = ChatOpenAI(temperature=0)
retriever = vectorstore.as_retriever()
tools = load_tools(
    ["llm-math"],
    llm = llm 
)

tools.append(
 create_retriever_tool(
    retriever,      #--> here is a RAG tool 
    "search_patient_record",
    "Searches and returns patient weight and physicals." # note the description of this tool. The agent will use this description to decide which tool to use!
 )
)

In [252]:
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)
result = agent_executor({"input": "How many liters of fluids my patient will receive if administered 30 mL/kg/hour IV infusion over 3 hours?"})

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-mGi0K4AmKu41HxKowsVRjoNh on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..




> Entering new AgentExecutor chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-mGi0K4AmKu41HxKowsVRjoNh on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-mGi0K4AmKu41HxKowsVRjoNh on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht


Invoking: `search_patient_record` with `patient_id`


[Document(page_content='A 28-year-old previously healthy adult patient presented with tachycardia, fever, and mental confusion. The symptoms started after a cut to his leg while gardening.', metadata={}), Document(page_content='body weight = 85 kg', metadata={}), Document(page_content='Temperature=99, Pulse=110, BP=100/60, Respiration=24, SpO2=92%', metadata={}), Document(page_content='redness, warmth, tenderness, and swelling of the skin', metadata={})]
Invoking: `Calculator` with `30 * 85 * 3`




Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-mGi0K4AmKu41HxKowsVRjoNh on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-mGi0K4AmKu41HxKowsVRjoNh on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

Answer: 7650

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-mGi0K4AmKu41HxKowsVRjoNh on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-mGi0K4AmKu41HxKowsVRjoNh on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

Your patient will receive 7650 mL of fluids if administered 30 mL/kg/hour IV infusion over 3 hours. To convert this to liters, divide by 1000:

7650 mL = 7.65 liters

> Finished chain.
